In [19]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
from numpy import trapz
from scipy.integrate import simps
import subprocess
import math

In [20]:
# creamos un array para que guarde los S/N y otro para los períodos de prueba

SN = np.empty(10)
trial_p = np.empty(10)

In [21]:
def signalnoise(tms):    # funcion que, dado un periodo, reduce y encuentra el S/N de la reduccion

# llamamos a prepfold

    filepref = "1907_" + str(tms)   # nombre (prefijo) del archivo

    subprocess.call("prepfold -topo -p " + str(tms/1000.) + " -n 128 -nosearch -o " + filepref + " 1907_truedm.dat", shell=True) 

# abrimos el archivo generado

    if x <= 283645:
        filename = filepref + "_283.64ms_Cand.pfd.bestprof"
    else:
        filename = filepref + "_283.65ms_Cand.pfd.bestprof"

# leemos los datos

    profbins = np.genfromtxt( filename, comments="none", dtype=int, skip_header=9, max_rows=1, usecols=(4) )
    profavg = np.genfromtxt( filename, comments="none", dtype=float, skip_header=10, max_rows=1, usecols=(4) )
    sigma = np.genfromtxt( filename, comments="none", dtype=float, skip_header=11, max_rows=1, usecols=(4) )

#   print("profbins_" + str(tms) + " = " + str(profbins))
    print("profavg_" + str(tms) + " = " + str(profavg))
#   print("sigma_" + str(tms) + " = " + str(sigma))
    
# Calculamos el área bajo la curva. Para ello, del archivo generado
# obtenemos una tabla Intensidad vs Fase (bines en realidad).

    data = np.loadtxt( filename, skiprows=26, usecols=(0,1) )

# Calculamos el off-pulse mean (profile baseline, Dc level)

    baseline = 0.
    k = 0
    for y in range( len(data[:,1]) ):
        if data[y,1] < profavg:
            baseline = baseline + data[y,1]
            k = k + 1
    
    base = baseline / float(k)
    print("base= " + str(base))
    
# Ploteamos el perfil

    prof_avg = np.full( (len(data[:,1])), profavg )      # Creamos un array que sólo contenga al profavg
    prof_base = np.full( (len(data[:,1])), base )     # Creamos un array que sólo contenga al baseline
    
    plt.suptitle(" p = " + str(tms) + " ms")
    plt.xlabel("Fase (bins)")
    plt.ylabel("Intensidad")
    plt.ylim(4.726e+09,4.738e+09)
    plt.plot(data)                                        # Graficamos el perfil
    plt.plot(data[:,0], prof_avg, label = "Prof Avg")     # Graficamos la línea horizontal del profavg
    plt.plot(data[:,0], prof_base, label = "Prof Base")   # Graficamos la línea horizontal del profavg
    plt.legend(loc="upper right")
    plt.savefig("profile2_" + str(tms) + ".png")
    plt.clf()
    
# Restamos el suelo a los datos

    data[:,1] = data[:,1] - base
    
# Calculamos el area usando la regla del trapecio y de Simpson

    areatrap = trapz(data[:,1] , dx=1)
    areasimps = simps(data[:,1] , dx=1)
    
#    print("areatrap= " + str(areatrap))
#    print("areasimps= " + str(areasimps))

# Calculamos el máximo del pulso

    peak = np.amax(data[:,1])

# Ahora calculamos W
# W is the equivalent width in bins of a top-hat pulse with the same area
# and peak height as the observed profile. W is simply the area under the
# observed pulse divided by its peak height.

    w = areasimps / peak

# Calculamos la sumatoria

    sum = 0
    for y in range( len(data[:,1]) ):
#        sum = sum + ( data[y,1] - base )
         sum = sum + ( data[y,1] )
    
#    print("w= " + str(w))
#    print("sum= " + str(sum))
    
# Calculamos la relación señal-ruido

    return ( 1. / ( sigma * math.sqrt(w) ) ) * sum    

In [22]:
# Set-up a period grid with values separated by 1 µs = 0.001 ms
# between 283.64 and 283.65 ms

i = 0
for x in range(283640, 283650, 1):
    
    ts = float(x)/1000000.          # time in seconds
    tms = float(x)/1000.            # time in miliseconds
    print("t[ms] = " + str(tms))
    
# calculamos la relacion señal ruido

    SN[i] = signalnoise(tms)
    trial_p[i] = tms
    print( "S/N_" + str(tms) + " = " + str(SN[i]) )
    print()
    
    i = i + 1
    
# graficamos S/N vs trial_p

    plt.xlabel("p")
    plt.ylabel("S/N")
    plt.plot(trial_p, SN)
    plt.savefig("sigalnoise.png")
    plt.clf()

# Hallamos el máximo S/N y el correspondiente período

print("the maximum S/N is: " + str( np.amax(SN) ))
print("the corresponding trial period is: " + str( trial_p[ np.where( SN == np.amax(SN)) ] ) ) 
    
    

t[ms] = 283.64
profavg_283.64 = 4726496294.18531
base= 4726209657.407408
S/N_283.64 = 154.80859100282504

t[ms] = 283.641
profavg_283.641 = 4726512957.90385
base= 4726226810.810811
S/N_283.641 = 166.42840227514918

t[ms] = 283.642
profavg_283.642 = 4726529621.62238
base= 4726240062.5
S/N_283.642 = 183.98682781876326

t[ms] = 283.643
profavg_283.643 = 4726546285.34091
base= 4726256596.491228
S/N_283.643 = 204.1149940367533

t[ms] = 283.644
profavg_283.644 = 4726562949.05945
base= 4726273793.103448
S/N_283.644 = 232.85664295589197

t[ms] = 283.645
profavg_283.645 = 4726579612.77798
base= 4726287387.931034
S/N_283.645 = 278.3469463496519

t[ms] = 283.646
profavg_283.646 = 4726596276.49652
base= 4726307110.169492
S/N_283.646 = 340.8094617544528

t[ms] = 283.647
profavg_283.647 = 4726612940.21505
base= 4726321358.974359
S/N_283.647 = 345.9687723400668

t[ms] = 283.648
profavg_283.648 = 4726629603.93358
base= 4726342508.474576
S/N_283.648 = 291.5185168889207

t[ms] = 283.649
profavg_283.649 

<Figure size 432x288 with 0 Axes>